In [1]:
#importing all libraries
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

#importing data
df = pd.read_csv('https://raw.githubusercontent.com/kelseytyler/219final/main/demographics1.csv')
locations = pd.read_csv('https://raw.githubusercontent.com/kelseytyler/219final/main/InNOut_Locations.csv')
whataburger = pd.read_csv('https://raw.githubusercontent.com/kelseytyler/219final/main/whataburger_2018_11_06.csv')[['city', 'state']].drop_duplicates()

In [2]:
#cleaning dataframe: breaking down race column so that each race has its own column
asian_counts = pd.Series([])
for x, y in df['Race'].items():
  if y == 'Asian':
    count = df['Count'].iloc[x]
    asian_counts[x] = count

df['Asian'] = asian_counts
df['Asian'].dropna()

white_counts = pd.Series([])
for x, y in df['Race'].items():
  if y == 'White':
    count = df['Count'].iloc[x]
    white_counts[x] = count

df['White'] = white_counts
df['White'].dropna()

hol_counts = pd.Series([])
for x, y in df['Race'].items():
  if y == 'Hispanic or Latino':
    count = df['Count'].iloc[x]
    hol_counts[x] = count

df['Hispanic or Latino'] = hol_counts
df['Hispanic or Latino'].dropna()

boa_counts = pd.Series([])
for x, y in df['Race'].items():
  if y == 'Black or African-American':
    count = df['Count'].iloc[x]
    boa_counts[x] = count

df['Black or African-American'] = boa_counts
df['Black or African-American'].dropna()
df.head(10)

AIorAN_counts = pd.Series([])
for x, y in df['Race'].items():
  if y == 'American Indian and Alaska Native':
    count = df['Count'].iloc[x]
    AIorAN_counts[x] = count

df['American Indian and Alaska Native'] = AIorAN_counts
df['American Indian and Alaska Native'].dropna()

#dropping all race columns except for white, leaving only one row for each city (not 5 duplicates)
white = df[df['Race'] == 'White'].drop(columns=['Asian', 'Hispanic or Latino', 'Black or African-American', 'American Indian and Alaska Native'])
white

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count,White
0,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402,76402.0
11,Waterbury,Connecticut,36.2,52235.0,56572.0,108807,3493.0,19967.0,2.71,CT,White,69075,69075.0
14,Carmichael,California,41.0,29281.0,33934.0,63215,4225.0,7378.0,2.39,CA,White,55862,55862.0
15,Daly City,California,39.7,53817.0,52757.0,106574,3782.0,56640.0,3.26,CA,White,25522,25522.0
22,Evanston,Illinois,36.8,34146.0,41377.0,75523,2058.0,15003.0,2.29,IL,White,54496,54496.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,Hillsboro,Oregon,35.3,50926.0,51434.0,102360,6625.0,19531.0,2.68,OR,White,81072,81072.0
2875,Bellevue,Washington,37.4,70763.0,69051.0,139814,5382.0,54654.0,2.43,WA,White,82378,82378.0
2884,South Bend,Indiana,32.4,50483.0,53274.0,103757,3708.0,8779.0,2.72,IN,White,71391,71391.0
2888,Oxnard,California,31.0,101906.0,105346.0,207252,6367.0,78678.0,4.08,CA,White,165423,165423.0


In [3]:
#creating columns for white pop. percent and minority pop. percent
white_perc = pd.Series([])
for x, y in white['White'].items():
  perc = y / white['Total Population'][x]
  white_perc[x] = perc
white['White Percentage'] = white_perc

minority_perc = pd.Series([])
for x, y in white['White Percentage'].items():
  minorityperc = 1 - y
  minority_perc[x] = minorityperc
white['Minority Percentage'] = minority_perc

#dropping unneeded columns and setting index to the cities
df = white.drop(columns = ['Race', 'Count', 'White'])
df.set_index('City')

,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,White Percentage,Minority Percentage
City,,,,,,,,,,,
Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,0.271013,0.728987
Waterbury,Connecticut,36.2,52235.0,56572.0,108807,3493.0,19967.0,2.71,CT,0.634840,0.365160
Carmichael,California,41.0,29281.0,33934.0,63215,4225.0,7378.0,2.39,CA,0.883683,0.116317
Daly City,California,39.7,53817.0,52757.0,106574,3782.0,56640.0,3.26,CA,0.239477,0.760523
Evanston,Illinois,36.8,34146.0,41377.0,75523,2058.0,15003.0,2.29,IL,0.721582,0.278418
...,...,...,...,...,...,...,...,...,...,...,...
Hillsboro,Oregon,35.3,50926.0,51434.0,102360,6625.0,19531.0,2.68,OR,0.792028,0.207972
Bellevue,Washington,37.4,70763.0,69051.0,139814,5382.0,54654.0,2.43,WA,0.589197,0.410803
South Bend,Indiana,32.4,50483.0,53274.0,103757,3708.0,8779.0,2.72,IN,0.688060,0.311940


In [4]:
#creating column where True = city has an In-N-Out, False = city doesn't have an In-N-Out
ino = pd.Series([])
all_cities = pd.Series([])

for x, y in df['City'].items():
  all_cities[x] = y

for x, y in locations['city_area'].items():
  ino[x] = y

matches = all_cities.isin(ino)

df['InNOut'] = matches

#splitting data into western states and eastern states
west = df[(df['State Code'] == 'CA') | (df['State Code'] == 'TX') | (df['State Code'] == 'OR')
 |(df['State Code'] == 'WA') |(df['State Code'] == 'CO') |(df['State Code'] == 'NV') |(df['State Code'] == 'ID') |
  (df['State Code'] == 'MT') |(df['State Code'] == 'AZ') |(df['State Code'] == 'NM') |(df['State Code'] == 'UT') |
  (df['State Code'] == 'WY')] #25% of InNOut column is True
east = (df[((df['State Code'] == 'ME') | (df['State Code'] == 'VA') | (df['State Code'] == 'NC')
 |(df['State Code'] == 'SC') |(df['State Code'] == 'GA') |(df['State Code'] == 'FL') |(df['State Code'] == 'PA') |
  (df['State Code'] == 'NY') |(df['State Code'] == 'DE') |(df['State Code'] == 'RI') |(df['State Code'] == 'CT') |
  (df['State Code'] == 'VT') |(df['State Code'] == 'MA') |(df['State Code'] == 'NH') |(df['State Code'] == 'MD')|
  (df['State Code'] == 'NJ')) & (df['City'] != 'Union City') &(df['City'] != 'Reading')]).dropna() #additionally drops duplicate rows and nans
west

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,White Percentage,Minority Percentage,InNOut
14,Carmichael,California,41.0,29281.0,33934.0,63215,4225.0,7378.0,2.39,CA,0.883683,0.116317,False
15,Daly City,California,39.7,53817.0,52757.0,106574,3782.0,56640.0,3.26,CA,0.239477,0.760523,True
27,Billings,Montana,36.3,52705.0,57565.0,110270,9121.0,3206.0,2.40,MT,0.929736,0.070264,False
38,Fullerton,California,34.5,69549.0,71300.0,140849,5394.0,43404.0,2.97,CA,0.654715,0.345285,True
53,Santa Ana,California,30.8,167503.0,167920.0,335423,4735.0,152999.0,4.58,CA,0.486240,0.513760,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2866,Tustin,California,33.4,39511.0,41052.0,80563,2434.0,25034.0,3.27,CA,0.492286,0.507714,True
2869,Vallejo,California,37.8,58379.0,62890.0,121269,8103.0,30592.0,2.83,CA,0.463177,0.536823,False
2872,Hillsboro,Oregon,35.3,50926.0,51434.0,102360,6625.0,19531.0,2.68,OR,0.792028,0.207972,False
2875,Bellevue,Washington,37.4,70763.0,69051.0,139814,5382.0,54654.0,2.43,WA,0.589197,0.410803,False
